In [ ]:
"""
Fetch photometric data for ZTF candidates and save as JSON files.

This script reads ZTF candidate coordinates from a CSV file, queries the SNAD API 
for photometry data, and stores the results in individual JSON files.
"""

import json
import os
import requests

import pandas as pd
from tqdm import tqdm

# Constants
ZTF_CANDIDATES_FILE = "/media3/rupesh/crp8/data/extragalactic_find_DR_photometry.csv" 
OUTPUT_SAVE_PATH = "/media3/rupesh/crp8/data/photometry_data" 
SEARCH_RADIUS_ARCSEC = 1.2
API_BASE_URL = "https://db.ztf.snad.space/api/v3/data/latest/circle/full/json"


def load_candidates(file_path):
    """Load ZTF candidates from a CSV file.

    Args:
        file_path (str): Path to the CSV file.

    Returns:
        pd.DataFrame: DataFrame containing ZTF candidate data.
    """
    return pd.read_csv(file_path)


def fetch_photometry_data(ra, dec, radius_arcsec=1.2):
    """Fetch photometry data from the SNAD API.

    Args:
        ra (float): Right ascension of the object.
        dec (float): Declination of the object.
        radius_arcsec (float): Search radius in arcseconds.

    Returns:
        dict: JSON response containing photometry data.
    """
    params = {
        "ra": ra,
        "dec": dec,
        "radius_arcsec": radius_arcsec
    }
    response = requests.get(API_BASE_URL, params=params)
    response.raise_for_status()
    return response.json()


def save_json(file_path, json_content):
    """Save JSON content to a file.

    Args:
        file_path (str): Path to save the JSON file.
        json_content (dict): JSON content to write.
    """
    with open(file_path, "w") as json_file:
        json.dump(json_content, json_file, indent=2)


def process_candidates(candidates_df, output_path):
    """Process each candidate and save corresponding photometry data.

    Args:
        candidates_df (pd.DataFrame): DataFrame of ZTF candidates.
        output_path (str): Directory to save JSON files.
    """
    for _, row in tqdm(candidates_df.iterrows(), total=len(candidates_df)):
        ra = row["mean_ra"]
        dec = row["mean_dec"]
        object_id = row["objectId"]

        try:
            data = fetch_photometry_data(ra, dec, SEARCH_RADIUS_ARCSEC)
            output_file = os.path.join(output_path, f"{object_id}.json")
            save_json(output_file, data)
        except requests.RequestException as e:
            print(f"Failed to fetch data for {object_id}: {e}")


def main():
    candidates = load_candidates(ZTF_CANDIDATES_FILE)
    os.makedirs(OUTPUT_SAVE_PATH, exist_ok=True)
    process_candidates(candidates, OUTPUT_SAVE_PATH)


if __name__ == "__main__":
    main()
